### Data Extraction from Stats Bomb

Geeting public information from Stats Bomb using public API


In [4]:
from statsbombpy import sb
import pandas as pd
from mplsoccer import VerticalPitch,Pitch
from highlight_text import ax_text, fig_text
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'statsbombpy'